初始状态下，点击所有位置Reward暂定为0

In [ ]:
from MineSweeper.core import MineSweeperService,MineSweeperGame,Cell,CellType
import hashlib

class RewardSingleData:
    def __init__(self, curGame : MineSweeperGame, clickX, clickY,reward = 0):
        self.curGame = curGame
        self.clickX = clickX
        self.clickY = clickY
        self.reward = reward
    
    def GetHashValue(self):
        return calculate_string_hash(self.curGame.GetGameDesString()+str(self.clickX)+str(self.clickY))

class RewardValueClass:
    def __init__(self,curRewardValue):
        self.curRewardValue = curRewardValue
        self.Count = 1

    def CalculateAverage(self,newRewardValue):
        self.curRewardValue = float(self.curRewardValue*self.Count + newRewardValue)/float(self.Count+1)
        self.Count += 1

class RewardData:
    def __init__(self):
        self.rewardDataDict = {}
    
    def AddRewardSingleData(self,rewardSingleData):
        curGameHashValue = rewardSingleData.GetHashValue()
        if self.rewardDataDict.get(curGameHashValue,None) is None:
            self.rewardDataDict.setdefault(curGameHashValue,RewardValueClass(rewardSingleData.reward))
        else:
            self.rewardDataDict[curGameHashValue].CalculateAverage(float(rewardSingleData.reward))
        
    def GetRewardValueDataCurRewardValue(self,rewardSingleData:RewardSingleData):
        curGameHashValue = rewardSingleData.GetHashValue()
        rewardValueClass : RewardValueClass = self.rewardDataDict.get(curGameHashValue,None)
        if  rewardValueClass is None:
            return False,None
        else:
            return True,rewardValueClass.curRewardValue

    def ClearRewardData(self):
        self.rewardDataDict.clear()

def calculate_string_hash(s):
    # 创建一个哈希对象
    hash_obj = hashlib.new("sha256")
    # 更新哈希对象的内容
    hash_obj.update(s.encode("utf-8"))
    # 返回十六进制格式的哈希值
    return hash_obj.hexdigest()


In [2]:
def CalculateRewardData(mineSweeperService:MineSweeperService,iter=100000):
    mineSweeperService = MineSweeperService(height=3,width = 3,mineNum = 3)
    rewardData = RewardData()
    for _ in range(iter):
        if mineSweeperService.IsGameOver():
            mineSweeperService.ReSetCurGame()
        mineSweeperService.RandomSafeClick()
        curGame = mineSweeperService.GetCurGame()
        for unRevealedCellIndex in curGame.UnRevealedSet:
            unRevealedCell = curGame.cellGrid.cells[unRevealedCellIndex[0]][unRevealedCellIndex[1]]
            unRevealedCell : Cell
            if unRevealedCell.cellType == CellType.Mine:
                reward = -2
            else:
                reward = 1
            rewardSingleData = RewardSingleData(curGame=curGame,
                                                clickX= unRevealedCell.pos[0],
                                                clickY=unRevealedCell.pos[1],
                                                reward=reward)
            rewardData.AddRewardSingleData(rewardSingleData=rewardSingleData)
    return rewardData

In [3]:
# 对于3 5^9*3*3 = 17,578,125 C(3,9) × A(6,9) = 5,080,320 实际上的情况 35714 ~ 30s
# 对于4 6^16*4*4 = 45,137,758,519,296 C(4,16) × A(12,16) = 1,586,644,899,840,000 1e4h 均匀采样
def CalculateStateNum(mineSweeperService:MineSweeperService,iter):
    mineSweeperService = MineSweeperService(3,3,3)
    rewardData = CalculateRewardData(mineSweeperService,iter)
    return len(rewardData.rewardDataDict)

#mineSweeperService = MineSweeperService(3,3,3)
#print(CalculateStateNum(mineSweeperService,int(1e4)))
#print(CalculateStateNum(mineSweeperService,int(1e5)))
#print(CalculateStateNum(mineSweeperService,int(1e6)))
#print(CalculateStateNum(mineSweeperService,int(1e7)))
#15387 33235 35714 35714


In [4]:
mineSweeperService = MineSweeperService(3,3,3)
rewardData = CalculateRewardData(mineSweeperService=mineSweeperService,iter=int(1e6))

In [5]:
#mineSweeperService = MineSweeperService(4,4,4)
#rewardData = CalculateRewardData(mineSweeperService=mineSweeperService,iter=1000000)

In [6]:
len(rewardData.rewardDataDict)

35714

In [7]:
#for rewarddataDictValue in rewardData.rewardDataDict.values():
#    print(rewarddataDictValue.curRewardValue)
#for rewarddataDictValue in rewardData.rewardDataDict.values():
#    print(rewarddataDictValue.Count)

In [8]:
'''
RewardMaxPolicy
'''
def RewardPWinRate(mineSweeperService:MineSweeperService,rewardData:RewardData,iter):
    GameRun = 0
    WinRun =0
    LossRun = 0
    
    for _ in range(iter):
        if mineSweeperService.IsGameStart():
            mineSweeperService.RandomClick()
        curGame = mineSweeperService.GetCurGame()
        maxClickX = None
        maxClickY = None
        curMaxReward = -2
        for pos in curGame.UnRevealedSet:
            clickX = pos[0]
            clickY = pos[1]
            curRewardSingleData = RewardSingleData(mineSweeperService.GetCurGame(),clickX,clickY,0)
            vaild,rewardValue = rewardData.GetRewardValueDataCurRewardValue(curRewardSingleData)
            if vaild:
                if rewardValue >= curMaxReward:
                    maxClickX = clickX
                    maxClickY = clickY
                    curMaxReward = rewardValue
            else:
                print("Never meet curGame")
                print(curGame.GetGameDesString())
                print(clickX,clickY)
        if maxClickX is None:
            #所有情况都没遇到过，取Set里第一个
            for pos in curGame.UnRevealedSet:
                maxClickX = pos[0]
                maxClickY = pos[1]
                break
        mineSweeperService.TryClick(maxClickX,maxClickY)
        if mineSweeperService.IsGameOver():
            if mineSweeperService.IsGameWin():
                WinRun += 1
            if mineSweeperService.IsGameLoss():
                LossRun += 1
            
            GameRun += 1
            mineSweeperService.ReSetCurGame()
            

    print(f"WinRun:{WinRun},GameRun:{GameRun},LossRun:{LossRun}")
    print("WinRate:",float(WinRun) / float(GameRun))

In [9]:
RewardPWinRate(mineSweeperService,rewardData,1000)

WinRun:175,GameRun:321,LossRun:146
WinRate: 0.5451713395638629


In [10]:
RewardPWinRate(mineSweeperService,rewardData,int(1e6))

WinRun:176686,GameRun:315699,LossRun:139013
WinRate: 0.5596660109788121
